In [16]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [17]:
class Neuron:
    def __init__(self, num_inputs):
        # Initialize weights and bias (randomly for demonstration)
        self.weights = torch.randn(num_inputs)
        self.bias = torch.tensor(0.0)

    def sigmoid(self, x):
        return 1 / (1 + torch.exp(-x))

    def forward(self, inputs):
        weighted_sum = torch.dot(self.weights, inputs) + self.bias
        output = self.sigmoid(weighted_sum)
        return output

In [18]:
neuron = Neuron(num_inputs=3)
neuron.weights = torch.tensor([0.5, -0.6, 0.2])
neuron.bias = 0.1

test_inputs = [
    torch.tensor([1.0, 0.0, 1.0]),
    torch.tensor([0.0, 1.0, 1.0]),
    torch.tensor([1.0, 1.0, 0.0]),
    torch.tensor([0.0, 0.0, 0.0]),
    torch.tensor([2.0, 3.0, 1.0])
]

for input in test_inputs:
    output = neuron.forward(input)
    print(f"Inputs: {input.numpy()}, Output: {output.item()}")

Inputs: [1. 0. 1.], Output: 0.6899744868278503
Inputs: [0. 1. 1.], Output: 0.4255574941635132
Inputs: [1. 1. 0.], Output: 0.5
Inputs: [0. 0. 0.], Output: 0.5249791741371155
Inputs: [2. 3. 1.], Output: 0.3775406777858734


### Adding a function to select the desired activation function

In [25]:
class Neuron:
    def __init__(self, num_inputs, activation):
        # Initialize weights and bias (randomly for demonstration)
        self.weights = torch.randn(num_inputs)
        self.bias = torch.tensor(0.0)
        self.activation_name = activation

    def activate(self, x):
        if self.activation_name == 'sigmoid':
            return 1 / (1 + torch.exp(-x))
        elif self.activation_name == 'relu':
            return torch.maximum(torch.tensor(0.0), x)
        elif self.activation_name == 'tanh':
            return torch.tanh(x)
        else:
            return x  # Linear activation

    def forward(self, inputs):
        weighted_sum = torch.dot(self.weights, inputs) + self.bias
        output = self.activate(weighted_sum)
        return output

In [29]:
activations = ['sigmoid', 'relu', 'tanh', 'linear']

for act in activations:
    neuron = Neuron(num_inputs=1, activation=act)
    neuron.weights = torch.tensor([1.0])
    neuron.bias = 0.0

    test_inputs = torch.tensor([2.0])
    output = neuron.forward(test_inputs)
    print(f"Activation: {act}, Input: {test_inputs.item()}, Output: {output:.4f}")

Activation: sigmoid, Input: 2.0, Output: 0.8808
Activation: relu, Input: 2.0, Output: 2.0000
Activation: tanh, Input: 2.0, Output: 0.9640
Activation: linear, Input: 2.0, Output: 2.0000


### Neurons to Layer

In [35]:
# Inputs                  Neuron 1                     Neuron 2
# -------                 --------                     --------
# x₁ = 2.0  ──(0.4)──┐
#                     ├─► ( Σ ) + 0.1 ──► 1.5
# x₂ = 3.0  ──(0.2)──┘


# x₁ = 2.0  ──(0.6)──┐
#                     ├─► ( Σ ) - 0.3 ──► 2.4
# x₂ = 3.0  ──(0.5)──┘


In [37]:
inputs = torch.tensor([2.0, 3.0]) # 2 inputs

weights = torch.tensor([[0.4, 0.6],
                       [0.2, 0.5]]) # weights for each of the 2 neurons

biases = torch.tensor([0.1, -0.3]) # biases for each of the 2 neurons

# Forward pass
outputs = inputs @ weights + biases

# Apply activation function
activation = torch.sigmoid(outputs)
print(f"Outputs after activation: {activation}")

Outputs after activation: tensor([0.8176, 0.9168])


### Complete layer with forward pass

In [38]:
class Layer:
    def __init__(self, num_inputs, num_neurons, activation='sigmoid'):
        self.num_neurons = num_neurons
        self.weights = torch.randn(num_inputs, num_neurons)
        self.biases = torch.zeros(num_neurons)
        self.activation_name = activation

    def activate(self, x):
        if self.activation_name == 'sigmoid':
            return 1 / (1 + torch.exp(-x))
        elif self.activation_name == 'relu':
            return torch.maximum(torch.tensor(0.0), x)
        elif self.activation_name == 'tanh':
            return torch.tanh(x)
        else:
            return x  # Linear activation

    def forward(self, inputs):
        weighted_sum = inputs @ self.weights.T + self.biases
        output = self.activate(weighted_sum)
        return output

In [ ]:
activations = ['sigmoid', 'relu', 'tanh', 'linear']
for act in activations:
    layer = Layer(num_inputs=2, num_neurons=3, activation=act)
    layer.weights = torch.tensor([[0.4, 0.6],
                                  [0.2, 0.5],
                                  [0.3, 0.9]])
    layer.biases = torch.tensor([0.1, -0.3, 0.2])
    inputs = torch.tensor([2.0, 3.0])
    output = layer.forward(inputs)
    print(f"Activation: {act}, Inputs: {inputs.numpy()}, Outputs: {output.numpy()}")
